<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/sensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum Sensing with Conjugate Pairs $\rho \otimes \rho*$**